In [1]:
import subprocess
import asyncio


async def run_script():
    # Crear un proceso para ejecutar el servidor sin bloquear el cuaderno
    process = await asyncio.create_subprocess_exec(
        'python3', 'tnt_api.py',
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE)

    try:
        # Salidas del programa
        stdout, stderr = await process.communicate()

        if process.returncode == 0:
            print("Script ejecutado exitosamente:\n", stdout.decode())
        else:
            print("Error en la ejecución del script:\n", stderr.decode())
    except asyncio.CancelledError:
        print("Tarea cancelada, cerrando el proceso.")
        process.terminate()
        await process.wait()
        raise


# Ejecutar la función en el event loop de asyncio
task = asyncio.create_task(run_script())


In [2]:
def stop_task():
    global task
    if task is not None:
        task.cancel()
        print("Cancelación de la tarea iniciada.")
    else:
        print("Tarea no encontrada")

Request de distritos -> Petición de todos los nombres de los distritos

In [2]:
import requests

response = requests.get('http://127.0.0.1:18000/distritos')
print(response.status_code)
print(response.text)

200
{"Distritos":["DESCONOCIDO","CENTRO","ARGANZUELA","RETIRO","SALAMANCA","CHAMARTÍN","TETUÁN","CHAMBERÍ","FUENCARRAL-EL PARDO","MONCLOA-ARAVACA","LATINA","CARABANCHEL","USERA","PUENTE DE VALLECAS","MORATALAZ","CIUDAD LINEAL","HORTALEZA","VILLAVERDE","VILLA DE VALLECAS","VICÁLVARO","SAN BLAS-CANILLEJAS","BARAJAS"]}


Recuperar el distrito con código 4 

In [3]:
response = requests.get('http://127.0.0.1:18000/distritos/4')
print(response.status_code)
print(response.text)

200
{"Distrito":"SALAMANCA"}


Insertar un radar

In [16]:
import requests

# URL de la API
url = 'http://127.0.0.1:18000/radares'

# Datos a enviar, asegúrate de que todos los campos requeridos estén incluidos
data = {
    'id': 999,
    'ubicacion': "Ubicación ejemplo",
    'carretera': "Carretera ejemplo",
    'punto_kilometrico': 42069.0,
    # Campos opcionales pueden ser omitidos o asignados a None si no se desea enviarlos
    'sentido': None,
    'tipo': None,
    'longitud': None,
    'latitud': None,
    'coordenadas': None,
    'cod_distrito': None
}

# Realizar la petición POST con datos JSON
response = requests.post(url, json=data)

# Imprimir la respuesta del servidor
print(response.text)


{"message":"Radar insertado con éxito","radar_id":999}


Eliminar el radar insertado

In [17]:
from my_connector import MySQLConnector

connector_helper = MySQLConnector(host="localhost", user="root", password="root", database="TNT")
connector_helper.connect()
query = """
DELETE FROM RadaresFijos where id = 999
"""
connector_helper.execute(query)
connector_helper.commit()
connector_helper.close()

Connected to MySQL database.
Query executed successfully.
Changes committed successfully.
Connection closed.


Insertar un accidente dummy para ser borrado

In [19]:
from my_connector import MySQLConnector

connector_helper = MySQLConnector(host="localhost", user="root", password="root", database="TNT")
connector_helper.connect()
query = """
Insert into Accidentes values (9999,'2022-01-03','10:00:00',"CALL. FRANCISCO DE ROJAS, 3",3,7,'Colisión lateral',440666.893,4475580.114,0,0,0
)
"""
connector_helper.execute(query)
connector_helper.commit()
connector_helper.close()

Connected to MySQL database.
Query executed successfully.
Changes committed successfully.
Connection closed.


Borrar el accidente dummy insertado

In [25]:
data = {
    'num_expediente': '9999'
}

response = requests.delete('http://127.0.0.1:18000/borrar_accidente', data=data)
print(response.status_code)
print(response.text)

422
{"detail":[{"loc":["body"],"msg":"value is not a valid dict","type":"type_error.dict"}]}
